# Bidirectional Encoder Representations from Transformers (BERT)
:label:`sec_bert`

我们已经介绍了几种用于自然语言理解的词嵌入模型。预训练后，输出可以被视为一个矩阵，其中每一行都是表示预定义词汇表中一个词的向量。实际上，这些词嵌入模型都是*上下文无关*的。让我们从说明这一特性开始。

## 从上下文无关到上下文相关

回想在:numref:`sec_word2vec_pretraining`和:numref:`sec_synonyms`中的实验。例如，word2vec和GloVe无论单词的上下文（如果有）如何，都会为相同的单词分配相同的预训练向量。形式上，任何标记$x$的上下文无关表示是一个只以$x$作为输入的函数$f(x)$。鉴于自然语言中多义性和复杂语义的丰富性，上下文无关表示有明显的局限性。例如，“crane”在“a crane is flying”和“a crane driver came”这两个上下文中具有完全不同的含义；因此，同一个词可能会根据上下文被分配不同的表示。

这促使了*上下文相关*词表示的发展，其中词的表示取决于它们的上下文。因此，标记$x$的上下文相关表示是依赖于$x$及其上下文$c(x)$的函数$f(x, c(x))$。流行的上下文相关表示包括TagLM（语言模型增强序列标注器）:cite:`Peters.Ammar.Bhagavatula.ea.2017`、CoVe（上下文向量）:cite:`McCann.Bradbury.Xiong.ea.2017`和ELMo（来自语言模型的嵌入）:cite:`Peters.Neumann.Iyyer.ea.2018`。

例如，通过将整个序列作为输入，ELMo是一个函数，它为输入序列中的每个词分配一个表示。具体来说，ELMo结合了预训练双向LSTM的所有中间层表示作为输出表示。然后，ELMo表示将作为附加特征添加到下游任务现有的监督模型中，例如通过连接ELMo表示和现有模型中的原始表示（如GloVe）。一方面，在添加ELMo表示后，预训练双向LSTM模型中的所有权重都被冻结。另一方面，现有的监督模型是专门为给定任务定制的。利用当时针对不同任务的最佳模型，添加ELMo改进了六个自然语言处理任务：情感分析、自然语言推理、语义角色标注、共指消解、命名实体识别和问答。

## 从任务特定到任务无关

尽管ELMo显著改进了多种自然语言处理任务的解决方案，但每个解决方案仍然依赖于*任务特定*架构。然而，为每个自然语言处理任务设计一个特定架构在实际操作中并不简单。GPT（生成式预训练）模型代表了一种设计用于上下文相关表示的通用*任务无关*模型的努力:cite:`Radford.Narasimhan.Salimans.ea.2018`。基于Transformer解码器，GPT预训练了一个将用于表示文本序列的语言模型。当将GPT应用于下游任务时，语言模型的输出将被送入一个新增的线性输出层来预测任务的标签。与ELMo冻结预训练模型参数形成鲜明对比的是，GPT在下游任务的监督学习过程中对预训练Transformer解码器中的*所有*参数进行微调。GPT在十二个自然语言推理、问答、句子相似度和分类任务上进行了评估，并且在九个任务上以最小的模型架构变化改进了最先进的水平。

然而，由于语言模型的自回归性质，GPT只能向前看（从左到右）。在“i went to the bank to deposit cash”和“i went to the bank to sit down”这两个上下文中，虽然“bank”对其左侧上下文敏感，GPT会为“bank”返回相同的表示，尽管它的含义不同。

## BERT：结合两者之长

正如我们所见，ELMo双向编码上下文但使用任务特定架构；而GPT是任务无关的，但仅从前向后编码上下文。结合两者的优点，BERT（来自Transformer的双向编码器表示）双向编码上下文，并且对于广泛范围内的自然语言处理任务只需要最小的架构更改:cite:`Devlin.Chang.Lee.ea.2018`。使用预训练的Transformer编码器，BERT能够基于其双向上下文表示任何标记。在下游任务的监督学习期间，BERT在两个方面类似于GPT。首先，BERT表示将被送入一个新增的输出层，根据任务的性质，模型架构的变化很小，例如预测每个标记与预测整个序列。其次，预训练Transformer编码器的所有参数都将被微调，而额外的输出层将从头开始训练。:numref:`fig_elmo-gpt-bert`描绘了ELMo、GPT和BERT之间的差异。

![A comparison of ELMo, GPT, and BERT.](../img/elmo-gpt-bert.svg)
:label:`fig_elmo-gpt-bert`

BERT进一步改进了十一项自然语言处理任务的最先进水平，涵盖了广泛的类别：(i) 单文本分类（例如，情感分析），(ii) 文本对分类（例如，自然语言推理），(iii) 问答，(iv) 文本标注（例如，命名实体识别）。从2018年提出的上下文相关的ELMo到任务无关的GPT和BERT，概念简单但经验强大的深度表示预训练彻底改变了各种自然语言处理任务的解决方案。

在本章的其余部分，我们将深入探讨BERT的预训练。当在:numref:`chap_nlp_app`解释自然语言处理应用时，我们将说明BERT在下游应用中的微调。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## [**输入表示**]
:label:`subsec_bert_input_rep`

在自然语言处理中，
一些任务（例如，情感分析）以单个文本作为输入，
而在其他一些任务（例如，自然语言推理）中，
输入是一对文本序列。
BERT 输入序列明确地表示了单个文本和文本对。
在前者中，
BERT 输入序列是特殊分类标记“&lt;cls&gt;”、
一个文本序列的标记以及特殊分隔标记“&lt;sep&gt;”的连接。
在后者中，
BERT 输入序列是“&lt;cls&gt;”、第一个文本序列的标记、
“&lt;sep&gt;”、第二个文本序列的标记以及“&lt;sep&gt;”的连接。
我们将始终区分术语“BERT 输入序列”
与其他类型的“序列”。
例如，一个 *BERT 输入序列* 可能包括一个 *文本序列* 或两个 *文本序列*。

为了区分文本对，
学习到的段嵌入 $\mathbf{e}_A$ 和 $\mathbf{e}_B$
分别添加到第一个序列和第二个序列的标记嵌入中。
对于单个文本输入，只使用 $\mathbf{e}_A$。

以下 `get_tokens_and_segments` 函数接受一个或两个句子
作为输入，然后返回 BERT 输入序列的标记
及其对应的段 ID。

In [2]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """Get tokens of the BERT input sequence and their segment IDs."""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0 and 1 are marking segment A and B, respectively
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

BERT 选择了 Transformer 编码器作为其双向架构。
在 Transformer 编码器中，
位置嵌入被添加到 BERT 输入序列的每个位置。
然而，与原始的 Transformer 编码器不同，
BERT 使用*可学习的*位置嵌入。
总而言之，:numref:`fig_bert-input` 显示
BERT 输入序列的嵌入是
词嵌入、段嵌入和位置嵌入的总和。

![The embeddings of the BERT input sequence are the sum
of the token embeddings, segment embeddings, and positional embeddings.](../img/bert-input.svg)
:label:`fig_bert-input`

以下 [**`BERTEncoder` 类**] 与 :numref:`sec_transformer` 中实现的 `TransformerEncoder` 类相似。
与 `TransformerEncoder` 不同，`BERTEncoder` 使用
段嵌入和可学习的位置嵌入。

In [3]:
#@save
class BERTEncoder(nn.Module):
    """BERT encoder."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                 num_blks, dropout, max_len=1000, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module(f"{i}", d2l.TransformerEncoderBlock(
                num_hiddens, ffn_num_hiddens, num_heads, dropout, True))
        # In BERT, positional embeddings are learnable, thus we create a
        # parameter of positional embeddings that are long enough
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # Shape of `X` remains unchanged in the following code snippet:
        # (batch size, max sequence length, `num_hiddens`)
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

假设词汇量为10000。
为了演示[**`BERTEncoder`的前向推理**]，
让我们创建它的实例并初始化其参数。

In [4]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
ffn_num_input, num_blks, dropout = 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                      num_blks, dropout)

我们将`tokens`定义为长度为8的2个BERT输入序列，其中每个token是词汇表中的一个索引。`BERTEncoder`对输入`tokens`进行前向推理，返回编码结果，其中每个token由一个向量表示，该向量的长度由超参数`num_hiddens`预定义。这个超参数通常被称为Transformer编码器的*隐藏大小*（隐藏单元的数量）。

In [5]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

## 预训练任务
:label:`subsec_bert_pretraining_tasks`

`BERTEncoder` 的前向推理给出了输入文本中每个标记以及插入的特殊标记“&lt;cls&gt;”和“&lt;seq&gt;”的BERT表示。接下来，我们将使用这些表示来计算预训练BERT的损失函数。预训练由以下两个任务组成：掩码语言模型和下一句预测。

### [**掩码语言模型**]
:label:`subsec_mlm`

如 :numref:`sec_language-model` 所示，语言模型使用其左侧的上下文来预测一个标记。为了在表示每个标记时编码双向上下文，BERT随机遮蔽标记，并以自监督的方式使用双向上下文中的标记来预测被遮蔽的标记。这个任务被称为*掩码语言模型*。

在这个预训练任务中，15%的标记将被随机选为被遮蔽的标记进行预测。为了在不利用标签的情况下预测一个被遮蔽的标记，一种直接的方法是在BERT输入序列中始终将其替换为特殊的“&lt;mask&gt;”标记。然而，这种人为的特殊标记“&lt;mask&gt;”在微调时永远不会出现。为了避免预训练和微调之间的这种不匹配，如果一个标记被遮蔽用于预测（例如，“great”在“this movie is great”中被选为遮蔽并预测），在输入中它将被替换为：

* 80%的时间用特殊的“&lt;mask&gt;”标记（例如，“this movie is great”变成“this movie is &lt;mask&gt;”）；
* 10%的时间用一个随机标记（例如，“this movie is great”变成“this movie is drink”）；
* 10%的时间保持不变的标签标记（例如，“this movie is great”变成“this movie is great”）。

注意，15%中有10%的时间会插入一个随机标记。这种偶尔的噪声鼓励BERT在其双向上下文编码中减少对被遮蔽标记（尤其是当标签标记保持不变时）的偏见。

我们实现以下 `MaskLM` 类来预测BERT预训练中的掩码语言模型任务中的被遮蔽标记。预测使用了一个单隐藏层的MLP (`self.mlp`)。在前向推理中，它接受两个输入：`BERTEncoder` 的编码结果和用于预测的标记位置。输出是这些位置上的预测结果。

In [6]:
#@save
class MaskLM(nn.Module):
    """The masked language model task of BERT."""
    def __init__(self, vocab_size, num_hiddens, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.LazyLinear(num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.LazyLinear(vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # Suppose that `batch_size` = 2, `num_pred_positions` = 3, then
        # `batch_idx` is `torch.tensor([0, 0, 0, 1, 1, 1])`
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

为了展示[**`MaskLM`的前向推理**]，我们创建其实例`mlm`并进行初始化。回想一下，来自`BERTEncoder`前向推理的`encoded_X`代表2个BERT输入序列。我们将`mlm_positions`定义为在`encoded_X`任一BERT输入序列中要预测的3个索引。`mlm`的前向推理返回`encoded_X`所有被遮蔽位置`mlm_positions`的预测结果`mlm_Y_hat`。对于每个预测，结果的大小等于词汇表的大小。

In [7]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

利用被遮罩的预测词元`mlm_Y_hat`的真实标签`mlm_Y`，我们可以计算BERT预训练中掩码语言模型任务的交叉熵损失。

In [8]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

### [**下一句预测**]
:label:`subsec_nsp`

尽管掩码语言模型能够编码双向上下文来表示词语，但它并没有显式地建模文本对之间的逻辑关系。为了帮助理解两个文本序列之间的关系，BERT在其预训练中考虑了一个二分类任务，即*下一句预测*。在生成用于预训练的句子对时，有一半的时间它们确实是连续的句子，并标记为“True”；而在另一半时间里，第二个句子是从语料库中随机抽取的，并标记为“False”。

下面的`NextSentencePred`类使用一个单隐藏层的MLP来预测在BERT输入序列中第二个句子是否是第一个句子的下一句。由于Transformer编码器中的自注意力机制，特殊标记“&lt;cls&gt;”的BERT表示编码了输入中的两个句子。因此，MLP分类器的输出层（`self.output`）将`X`作为输入，其中`X`是MLP隐藏层的输出，该隐藏层的输入是编码后的“&lt;cls&gt;”标记。

In [9]:
#@save
class NextSentencePred(nn.Module):
    """The next sentence prediction task of BERT."""
    def __init__(self, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.LazyLinear(2)

    def forward(self, X):
        # `X` shape: (batch size, `num_hiddens`)
        return self.output(X)

我们可以看到，[**`NextSentencePred`的前向推理**]实例对每个BERT输入序列返回二元预测。

In [10]:
# PyTorch by default will not flatten the tensor as seen in mxnet where, if
# flatten=True, all but the first axis of input data are collapsed together
encoded_X = torch.flatten(encoded_X, start_dim=1)
# input_shape for NSP: (batch size, `num_hiddens`)
nsp = NextSentencePred()
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

这两个二分类的交叉熵损失也可以计算。

In [11]:
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

值得注意的是，上述所有预训练任务中的标签都可以从预训练语料库中轻易获得，无需人工标注。原始的BERT是在BookCorpus :cite:`Zhu.Kiros.Zemel.ea.2015`和英文维基百科的合并文本上进行预训练的。这两个文本语料库非常庞大：它们分别包含8亿词和25亿词。

## [**综合起来**]

在预训练BERT时，最终的损失函数是掩码语言模型和下一句预测两个损失函数的线性组合。现在我们可以通过实例化`BERTEncoder`、`MaskLM`和`NextSentencePred`这三个类来定义`BERTModel`类。前向推理返回编码后的BERT表示`encoded_X`、掩码语言模型的预测`mlm_Y_hat`以及下一句预测`nsp_Y_hat`。

In [12]:
#@save
class BERTModel(nn.Module):
    """The BERT model."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens,
                 num_heads, num_blks, dropout, max_len=1000):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens,
                                   num_heads, num_blks, dropout,
                                   max_len=max_len)
        self.hidden = nn.Sequential(nn.LazyLinear(num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens)
        self.nsp = NextSentencePred()

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # The hidden layer of the MLP classifier for next sentence prediction.
        # 0 is the index of the '<cls>' token
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat

## 摘要

* 诸如word2vec和GloVe之类的词嵌入模型是上下文无关的。无论单词的上下文如何（如果有），它们都会为相同的单词分配相同的预训练向量。对于处理自然语言中的多义性或复杂语义来说，这很困难。
* 对于像ELMo和GPT这样的上下文敏感的词表示方法，词语的表示取决于其上下文。
* ELMo双向编码上下文，但使用特定任务架构（然而，为每个自然语言处理任务设计一个特定架构实际上并不容易）；而GPT与任务无关，但仅从左到右编码上下文。
* BERT结合了两者的优点：它双向编码上下文，并且在广泛的自然语言处理任务中只需要最小的架构更改。
* BERT输入序列的嵌入是词嵌入、段嵌入和位置嵌入的总和。
* BERT的预训练由两个任务组成：掩码语言建模和下一句预测。前者能够为表示词语编码双向上下文，而后者则明确地对文本对之间的逻辑关系进行建模。

## 练习

1. 在其他条件相同的情况下，掩码语言模型相比于从左到右的语言模型，在收敛时需要更多的还是更少的预训练步骤？为什么？
1. 在BERT的原始实现中，`BERTEncoder`（通过`d2l.TransformerEncoderBlock`）中的逐位置前馈网络和`MaskLM`中的全连接层都使用高斯误差线性单元（GELU）:cite:`Hendrycks.Gimpel.2016`作为激活函数。研究GELU与ReLU之间的差异。

[讨论](https://discuss.d2l.ai/t/1490)